In [9]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json

In [10]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m

In [11]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [12]:
gdf_geom : gpd.GeoSeries = gdf.boundary
# 30s

In [13]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m

In [14]:
gl = gdml.apply(lambda x: x.geoms if x is not None else None).explode()
# 1m

In [15]:
gl.dropna(inplace=True)

In [16]:
coords = gl.apply(lambda x: list(x.coords))
# 1m 30s

In [17]:
segments = coords.apply(lambda x: list([x[i:i+2] for i in range(len(x)-1)])).explode()
# 4m

In [18]:
segments = segments.tolist()

In [21]:
with open('local/segments.json', 'w') as f:
    json.dump(segments, f)
# 10m